<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #5: Add new frames to a Zooniverse workflow</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Nov 29th, 2021</h5>

# Set up and requirements

### Import Python packages

In [4]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t5_utils as t5
import kso_utils.zooniverse_utils as zoo

print("Packages loaded successfully")

<IPython.core.display.Javascript object>

Packages loaded successfully


### Choose your project

In [5]:
project = t_utils.choose_project()

Dropdown(description='Project:', options=('Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'), value='Koster_Sea…

### Initiate sql and zoo project

In [21]:
# Initiate db
db_info_dict = t_utils.initiate_db(project.value)

Enter your username for SNIC server········
Enter your password for SNIC server········
Updated sites
Updated movies
Updated species


In [7]:
# Connect to Zooniverse project
zoo_project = t_utils.connect_zoo_project(project.value)

Enter your Zooniverse user········
Enter your Zooniverse password········


# Retrieve info about zooniverse clips

In [8]:
zoo_info_dict = t_utils.retrieve__populate_zoo_info(project_name = project.value, 
                                                    db_info_dict = db_info_dict,
                                                    zoo_project = zoo_project,
                                                    zoo_info = ["subjects"])

Retrieving subjects from Zooniverse
subjects were retrieved successfully
Updated subjects
The database has a total of 2342 frame subjects and 7362 clip subjects have been updated


# Retrieve frames

## Select the species of interest

In [9]:
# Specify the species of interest
species_i = t5.choose_species(db_info_dict["db_path"])

SelectMultiple(description='Species', index=(0,), options=('Angular crab', 'Ascidians (any species)', 'Bivalve…

### Specify the way frames will be accessed

In [22]:
frame_method = t5.select_frame_method()

Combobox(value='', description='Select frame method:', ensure_option=True, options=('Manual', 'Automatic (from…

### Create the frames

In [ ]:
frames_to_upload_df = t5.get_frames(species_ids = species_i, 
                                     db_path = db_info_dict["db_path"],
                                     db_info_dict = db_info_dict,
                                     frame_method = frame_method.value)

## List available frames to upload to Zooniverse

In [ ]:
# Specify the server to connect to
server_i = t_utils.choose_server()

In [ ]:
# Check availability of movies that correspond to the aggregated clips
t_utils.connect_server(server_i.value)

In [ ]:
#Specify how many frames per clip and how many clips of species i to upload


In [ ]:
# Set the subject_set to upload the frames to


### Preview the frames

In [ ]:
# Upload the frames to the project


### Set Zooniverse metadata

In [ ]:
upload_to_zoo, sitename, created_on = t5.set_zoo_metadata(df = clips_ready_df,
                                                          project_name = project.value)

#upload_to_zoo = upload_to_zoo.fillna("unknown")



### Upload frames to Zooniverse

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

In [ ]:
t5.upload_frames_to_zooniverse(upload_to_zoo = upload_to_zoo, 
                              sitename = sitename,
                              created_on = created_on,
                              project = zoo_clips_info_dict["project"])

#END